In [20]:
import os

In [21]:
%pwd

'/Users/alamgirkabir/chicken-diease-classification'

In [4]:
os.chdir('../')

In [5]:
%pwd

'/Users/alamgirkabir/chicken-diease-classification'

In [22]:
import tensorflow as tf

In [8]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [23]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [24]:
from chickenDieaseClassifier.constants import *
from chickenDieaseClassifier.utils.common import read_yaml, create_directories, save_json
import pandas as pd

In [35]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = Path(self.config.training.trained_model_path),
            training_data = Path(self.config.data_ingestion.unzip_dir),
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE  
        )
        return eval_config

In [34]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):
        data_df = pd.read_csv(os.path.join(self.config.training_data,'train_data.csv'),dtype = str)


        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_dataframe(
            dataframe=data_df,
            directory=os.path.join(self.config.training_data,'Train'),
            x_col='images',
            y_col = 'label',
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


In [36]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(config=eval_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

[2024-01-07 12:55:24,210: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-07 12:55:24,225: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-07 12:55:24,239: INFO: common: created directory at: artifacts]


Found 2420 validated image filenames belonging to 4 classes.
152/152 [==============================] - 915s 6s/step - loss: 4.5065 - accuracy: 0.6888
[2024-01-07 13:10:41,089: INFO: common: json file saved at scores.json]
